In [1]:
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dotenv import load_dotenv
from toolkits.datapreparing import download_monthly_tables, collect_data
from toolkits.datasets import CNNDataset, train_test_split, load_next_5min
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import feather
import os
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

c:\Users\Home\anaconda3\envs\route-plan\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## main

# download_monthly_tables(start='2023-01-01', end='2023-12-31', dest_dir='./nfb2023', file_format='feather')
speedCollection, volCollection, occCollection, laneCollection, tunnelCollection = collect_data()

trainSpeed, trainVol, trainOcc, trainNumLane, trainTunnel,\
testSpeed, testVol, testOcc, testNumLane, testTunnel =\
    train_test_split(speedCollection, volCollection, occCollection, laneCollection, tunnelCollection, test_size=0.2)

trainDataset = CNNDataset(speed_data=trainSpeed, volume_data=trainVol, occupy_data=trainOcc,
                          lane_data=trainNumLane, tunnel_data=trainTunnel, load_ckpt=False, mode='train')
testDataset = CNNDataset(speed_data=testSpeed, volume_data=testVol, occupy_data=testOcc,
                         lane_data=testNumLane, tunnel_data=testTunnel, load_ckpt=False, mode='test')

In [9]:
trainDataset[0][0]

tensor([[[0.7050, 0.8420, 0.7810, 0.8230, 0.6750, 0.8050],
         [0.8010, 0.7880, 0.8160, 0.8040, 0.7190, 0.6900],
         [0.8680, 0.8480, 0.8790, 0.8620, 0.7550, 0.6620]],

        [[0.0062, 0.0055, 0.0073, 0.0067, 0.0047, 0.0077],
         [0.0060, 0.0055, 0.0070, 0.0068, 0.0047, 0.0073],
         [0.0065, 0.0070, 0.0058, 0.0062, 0.0048, 0.0082]],

        [[0.0270, 0.0270, 0.0370, 0.0320, 0.0230, 0.0370],
         [0.0280, 0.0260, 0.0320, 0.0330, 0.0200, 0.0360],
         [0.0280, 0.0330, 0.0250, 0.0280, 0.0220, 0.0400]],

        [[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]]])

### Directly load dataset

You can also load datasets from `.h5` file if you have saved them.

In [ ]:
trainDataset = CNNDataset(load_ckpt=True, mode='train', ckpt_dir='./toolkits/cnndataset')
testDataset = CNNDataset(load_ckpt=True, mode='test', ckpt_dir='./toolkits/cnndataset')

In [ ]:
trainDataset.tunnelFeature[0]

In [ ]:
trainDataset[0]

Create dataloaders

## Define Neural Network Architecture

In [ ]:
class CNNRegression(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cnnLayer = nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=16, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),
        )
        
        self.fcLayer = nn.Sequential(
            nn.Linear(32 * 1 * 4, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(self.device)

    def forward(self, x) -> torch.Tensor:
        x = self.cnnLayer(x)
        x = x.flatten(1)
        x = self.fcLayer(x)
        return x

In [ ]:
# Hyperparams for training
batch_size = 256
lr = 1e-3
n_epochs = 200

# Prepare datasets and dataloaders
trainDataset, testDataset = load_next_5min()
trainLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
testLoader = DataLoader(testDataset, batch_size=batch_size, shuffle=False)

model = CNNRegression()
optimizer = optim.Adam(params=model.parameters(), lr=lr, weight_decay=1e-8)

In [ ]:
for epoch in range(n_epochs):
    # Switch to train mode
    model.train()
    
    # Record Info in training
    train_loss = []

    for batch in tqdm(trainLoader):
        X, y = batch
        logits = model(X.to(model.device))
        loss = F.mse_loss(logits, y.to(model.device))
        
        # Compute gradients and update model params
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

    # Compute the average train_loss
    train_loss = sum(train_loss) / len(train_loss)
    print(f"[ Train | {epoch + 1:d}/{n_epochs:d} ] loss = {train_loss:.5f}")